# 일반 fine-tuning과 lora 모델 활용 학습의 성능 차이 비교


### 1. 환경 설정

In [ ]:
!pip install -q transformers datasets peft accelerate torch scikit-learn

In [ ]:
import torch
print(f"GPU 사용 가능: {torch.cuda.is_available()}")
print(f"현재 GPU: {torch.cuda.get_device_name(0)}" if torch.cuda.is_available() else "GPU를 사용할 수 없습니다.")

### 2. 데이터셋 로드 및 전처리

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# KLUE ynat 데이터셋 로드
dataset = load_dataset("klue", "ynat")

# 라벨 정보 확인
print(dataset['train'].features['label'].names)

In [ ]:
from datasets import load_dataset
klue_tc_train = load_dataset('klue', 'ynat', split='train')
klue_tc_eval = load_dataset('klue', 'ynat', split='validation')
klue_tc_train

In [ ]:
# 실습에 사용하지 않는 불필요한 컬럼 제거
klue_tc_train = klue_tc_train.remove_columns(['guid', 'url', 'date'])
klue_tc_eval = klue_tc_eval.remove_columns(['guid', 'url', 'date'])
klue_tc_train

In [ ]:
# 카테고리를 문자로 표기한 label_str 컬럼 추가

klue_tc_train.features['label']
# ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)

klue_tc_train.features['label'].int2str(1)
# '경제'

klue_tc_label = klue_tc_train.features['label']

def make_str_label(batch):
  batch['label_str'] = klue_tc_label.int2str(batch['label'])
  return batch

klue_tc_train = klue_tc_train.map(make_str_label, batched=True, batch_size=1000)

klue_tc_train[0]

In [ ]:
model_id = "klue/roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=len(train_dataset.features['label'].names)  # 분류할 라벨 수를 자동 설정
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# 학습/검증/테스트 데이터셋 분할
train_dataset = klue_tc_train.train_test_split(test_size=10000, shuffle=True, seed=42)['test']
dataset = klue_tc_eval.train_test_split(test_size=1000, shuffle=True, seed=42)
test_dataset = dataset['test']
valid_dataset = dataset['train'].train_test_split(test_size=1000, shuffle=True, seed=42)['test']

In [ ]:
# 토크나이징 함수 정의
def tokenize_function(batch):
    return tokenizer(batch["title"], padding="max_length", truncation=True, max_length=128)

# 전처리
train_tokenized = train_dataset.map(tokenize_function, batched=True)
valid_tokenized = valid_dataset.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)

### 3. 일반 Fine-Tuning

In [ ]:
import torch
import numpy as np
from transformers import (
    Trainer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    AutoTokenizer
)

# ✅ 모델 학습 설정 정의
training_args = TrainingArguments(
    output_dir="./results",           # 모델과 로그 저장 디렉토리
    num_train_epochs=1,               # 전체 학습 에폭 수
    per_device_train_batch_size=8,    # GPU/CPU 하나당 학습 배치 사이즈
    per_device_eval_batch_size=8,     # GPU/CPU 하나당 평가 배치 사이즈
    eval_strategy="epoch",            # 에폭 끝날 때마다 평가 수행
    learning_rate=5e-5,               # 학습률
    push_to_hub=False,                # 모델 결과를 허깅페이스 허브에 업로드하지 않음
    report_to="none"                  # 학습 레포트를 전송하지 않음
)

# ✅ 평가 지표 함수 정의 (정확도만 계산)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)   # 가장 높은 확률을 가진 클래스 선택
    return {"accuracy": (predictions == labels).mean()}  # 정확도 계산


In [ ]:
# ✅ Trainer 객체 생성
trainer = Trainer(
    model=model,                         # 사전학습된 분류 모델
    args=training_args,                  # 학습 설정
    train_dataset=train_tokenized,         # 학습 데이터셋
    eval_dataset=valid_tokenized,          # 검증 데이터셋
    tokenizer=tokenizer,                 # 토크나이저 (로깅, 오류 방지용)
    compute_metrics=compute_metrics,     # 평가 지표 함수
)

# ✅ 모델 학습 시작
trainer.train()

# ✅ 테스트 데이터셋에 대한 평가 수행
trainer.evaluate(test_tokenized)

In [ ]:
from transformers import pipeline

# 모델 저장
model.save_pretrained("./results")
tokenizer.save_pretrained("./results")

# 이미 파인튜닝된 모델 로드
model_path = "./results"
classifier = pipeline("text-classification", model=model_path, tokenizer="klue/roberta-base")

# 실제 라벨 이름 매핑
label_map = {
    "LABEL_0": "IT과학",
    "LABEL_1": "경제",
    "LABEL_2": "사회",
    "LABEL_3": "생활문화",
    "LABEL_4": "세계",
    "LABEL_5": "스포츠",
    "LABEL_6": "정치"
}

# 테스트 문장
test_texts = [
    "올해 외국인 주식 투자 성적, 개미 수익률의 4배로 압승.",  # 경제
    "성적은 4위 하지만 인기는 1위 삼성라이온즈, 2025프로야구 관중 순위, 82.5%라는 미친 좌석 점유율.",  # 스포츠
    "대통령 '한글이 그린 세상, 국민이 주인인 나라의 또 다른 모습'.",  # 정치
    "AI 붐타고 '제 2전성기' 맞은 이 회사... 클라우드 업계 엔비디아 될 수 있을까",  # IT과학
    "고속도로 실시간 교통상황 '부산->서울 6시간 40분'... 오후 4~5시 절정.",  # 생활문화
    "10대 사망 최대 원인은 '자살'...10대도 심리부검 검토해야"  # 사회
]

# 예측 실행
for text in test_texts:
    result = classifier(text)[0]
    label_name = label_map[result["label"]]
    print(f"📰 입력 문장: {text}")
    print(f"📌 예측 결과: {label_name} ({result['score']:.2f})")
    print("-" * 60)

In [ ]:
import torch, os, shutil
from pathlib import Path

print("===== Fine-tuned Model Summary =====")

result = trainer.evaluate(valid_tokenized)
val_acc = result.get("eval_accuracy", result.get("accuracy"))
print(f"Validation Accuracy : {val_acc:.4f}")

train_time = None
if hasattr(trainer, "state") and trainer.state is not None:
    train_time = getattr(trainer.state, "train_runtime", None)
    if train_time is None and hasattr(trainer.state, "log_history"):
        # log_history에서 가장 최근의 train_runtime 탐색
        for rec in reversed(trainer.state.log_history):
            if isinstance(rec, dict) and "train_runtime" in rec:
                train_time = rec["train_runtime"]
                break

# 없다면 알 수 없음 처리
if train_time is None:
    train_time = float("nan")

if not np.isnan(train_time):
    print(f"Training Time       : {train_time:.2f} sec ({train_time/60:.2f} min)")
else:
    print("Training Time        : N/A")

gpu_mem = torch.cuda.max_memory_allocated() / 1e9 if torch.cuda.is_available() else 0
print(f"GPU 메모리 사용량   : {gpu_mem:.2f} GB")

save_dir = "./saved_model"
if os.path.exists(save_dir): shutil.rmtree(save_dir)
trainer.save_model(save_dir)
model_size = sum(f.stat().st_size for f in Path(save_dir).rglob("*")) / (1024**2)
print(f"모델 저장 용량      : {model_size:.2f} MB")


### 4. LoRA Fine-Tuning

In [ ]:
# 메모리 초기화
import torch, gc, os, time, shutil
from pathlib import Path

gc.collect()
torch.cuda.empty_cache()
print("GPU 메모리 초기화 완료")

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np

model_id = "klue/roberta-base"

# 사전 학습된 KLUE Roberta 분류 모델 불러오기
model_lora = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=7
)

# LoRA 설정
lora_config = LoraConfig(
    r=8,                    # 랭크
    lora_alpha=16,          # 스케일링 계수
    lora_dropout=0.1,       # 드롭아웃 비율
    bias="none",
    task_type="SEQ_CLS",    # 분류 태스크용
    target_modules=["query", "value"],  # attention 부분에 적용
)

# 모델에 LoRA 적용
model_lora = get_peft_model(model_lora, lora_config)
print("\nLoRA 모델의 학습 가능한 파라미터 수:")
model_lora.print_trainable_parameters()

print("KLUE RoBERTa 모델에 LoRA 적용 완료")

In [ ]:
#Trainer 설정 (일반 fine-tuning과 동일)

training_args_lora = TrainingArguments(
    output_dir="./results_lora",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    learning_rate=2e-4,             # LoRA는 좀 더 큰 학습률 가능
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": (preds == labels).mean()}

trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=train_tokenized,    # 기존과 동일한 학습 데이터
    eval_dataset=valid_tokenized,     # 동일한 검증 데이터
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# LoRA 학습 및 평가

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

start = time.time()
train_output = trainer_lora.train()
end = time.time()

eval_metrics = trainer_lora.evaluate(eval_dataset=valid_tokenized)

In [ ]:
val_acc = eval_metrics.get("eval_accuracy", eval_metrics.get("accuracy"))
train_time = train_output.metrics.get("train_runtime", end - start)
gpu_mem = torch.cuda.max_memory_allocated() / 1e9 if torch.cuda.is_available() else 0

save_dir = "./saved_model_lora"
if os.path.exists(save_dir):
    shutil.rmtree(save_dir)
trainer_lora.save_model(save_dir)
model_size = sum(f.stat().st_size for f in Path(save_dir).rglob("*")) / (1024**2)

print("===== LoRA Fine-tuning Summary =====")
print(f"Validation Accuracy : {val_acc:.4f}")
print(f"Training Time       : {train_time:.2f} sec ({train_time/60:.2f} min)")
print(f"GPU 메모리 사용량   : {gpu_mem:.2f} GB")
print(f"모델 저장 용량      : {model_size:.2f} MB")